<a href="https://colab.research.google.com/github/skyjadel/chang-s_titanic/blob/main/Titanic_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

In [2]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df = df[list(set(df.columns).difference(['Ticket','Name','Cabin']))]
df

,Age,Parch,Sex,SibSp,Pclass,PassengerId,Survived,Fare,Embarked
0,22.0,0,male,1,3,1,0,7.2500,S
1,38.0,0,female,1,1,2,1,71.2833,C
2,26.0,0,female,0,3,3,1,7.9250,S
3,35.0,0,female,1,1,4,1,53.1000,S
4,35.0,0,male,0,3,5,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,27.0,0,male,0,2,887,0,13.0000,S
887,19.0,0,female,0,1,888,1,30.0000,S
888,NaN,2,female,1,3,889,0,23.4500,S
889,26.0,0,male,0,1,890,1,30.0000,C


# 資料前處理

## 區分 Training Set 與 Testing Set

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=109)
print(f'{len(train_df)} rows in training set.')
print(f'{len(test_df)} rows in testing set.')

712 rows in training set.
179 rows in testing set.


## 補缺失值

從上次的EDA我們知道，有缺失值的欄位有 Age 和 Embarked，Age將補上training set 的平均值，Embarked將補上trainin set 的最多分類。

In [5]:
# 找出哪個港口上船的人最多
grouped = train_df.groupby('Embarked')
harbor_counts = {g[0]:len(g[1]) for g in grouped}

max_count = 0
for k, v in harbor_counts.items():
  if v > max_count:
    max_count = v
    major_embarked_harbor = k

In [6]:
# Training Set 的年齡平均值
#mean_age = train_df['Age'].mean()
mean_age = {i: train_df[train_df['Pclass']==i]['Age'].mean() for i in range(1,4)}

In [7]:
def fill_nan(df, age_value=mean_age, embarked_value=major_embarked_harbor):
  df = df.fillna({'Embarked':embarked_value})


  if type(age_value) == dict:
    age_is_na = df['Age'].isna()
    for i in df.index:
      if age_is_na[i]:
        df.loc[i, 'Age'] = mean_age[df.loc[i, 'Pclass']]
  else:
    df = df.fillna({'Age':age_value})

  return df

## Labeling

Sex 欄位改稱 Sex_female 欄位 <br>
Embarked 使用onehot

In [8]:
def labeling(df):
  df = pd.concat([df, pd.get_dummies(df[['Sex', 'Embarked']])], axis=1)
  df.drop(['Sex', 'Embarked', 'Sex_male'], axis=1, inplace=True)
  return df

## 前處理函式

In [9]:
def preprocess(df):
  return labeling(fill_nan(df))

In [10]:
train_df = preprocess(train_df)
test_df = preprocess(test_df)

In [11]:
train_df

,Age,Parch,SibSp,Pclass,PassengerId,Survived,Fare,Sex_female,Embarked_C,Embarked_Q,Embarked_S
70,32.000000,0,0,2,71,0,10.5000,False,False,False,True
183,1.000000,1,2,2,184,1,39.0000,False,False,False,True
599,49.000000,0,1,1,600,1,56.9292,False,True,False,False
815,38.637973,0,0,1,816,0,0.0000,False,False,False,True
674,29.525926,0,0,2,675,0,0.0000,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
399,28.000000,0,0,2,400,1,12.6500,True,False,False,True
141,22.000000,0,0,3,142,1,7.7500,True,False,False,True
757,18.000000,0,0,2,758,0,11.5000,False,False,False,True
245,44.000000,0,2,1,246,0,90.0000,False,False,True,False


# 訓練

In [12]:
columns_X = list(set(train_df.columns) - {'Survived', 'PassengerId'})
columns_y = ['Survived']

train_X = train_df[columns_X]
train_y = train_df[columns_y]
test_X = test_df[columns_X]
test_y = test_df[columns_y]

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)
print(f'平均準確率{np.mean(scores):.4f}')

[0.73426573 0.83216783 0.84507042 0.73239437 0.87323944]
平均準確率0.8034


In [16]:
H = log.fit(np.array(train_X), np.array(train_y).reshape(-1))

In [19]:
pred_y = log.predict(np.array(train_X))
true_y = yt = np.array(train_y).reshape(-1)
acc_samples = sum(true_y * pred_y) + sum((1-true_y)*(1-pred_y))
print(f'Accuracy for training set is {acc_samples/len(train_df):.4f} ({acc_samples}/{len(train_df)})')

Accuracy for training set is 0.8118 (578/712)


In [17]:
pred_y = log.predict(np.array(test_X))
true_y = yt = np.array(test_y).reshape(-1)
acc_samples = sum(true_y * pred_y) + sum((1-true_y)*(1-pred_y))
print(f'Accuracy for testing set is {acc_samples/len(test_df):.4f} ({acc_samples}/{len(test_df)})')

Accuracy for testing set is 0.8101 (145/179)
